# parseia-json
Esse script transforma os dados do CSV, já filtrado, manualmente checado e com apenas os discursos que mencionam a ditadura, e transforma em um formato .json para gerar a visualização de dados.

#### Importar pacotes

In [22]:
import json
import pandas as pd
import pprint as pp

#### Formatar dados ao longo do tempo

In [23]:
def resample_df(df):
    
    '''
    Essa função recebe o dataframe que veio de 'buscador-da-ditadura.ipynb',
    mas com duas colunas extras ("HIGHLIGHT" e "IS_MATCH") que indicam,
    respectivamente, o trecho para ser destacado no interativo e se a
    entrada realmente menciona a ditadura militar.
    '''
    
    # Filtra para manter os anos que temos inteiros
    df = df['2001-01-01':'2018-12-31']
    
    # Baseado em: stackoverflow.com/questions/45908146/

    # Cria um índice no intervalo desejado
    t_index = pd.DatetimeIndex(start='2001-01-01', end='2018-12-31', freq='M')

    # Faz o resample e agregação
    df = df.resample('M').agg({
        # Primeiro, contamos quantos discursos foram registrados no mês
        "FILE":'count', 
        # Depois, quantos deles mecionaram a ditadura.
        # Soma funciona porque, quando há uma menção, o valor é 1. Quando não é, o valor é zero
        "IS_MATCH":'sum'
    })

    # Aplica no novo índice
    df = df.reindex(t_index)

    # Reseta índice
    df = df.reset_index()

    # Renomeia colunas
    df = df.rename( columns = {
        'index' : "SESSION_DATE",
        'FILE'  : 'ALL_MENTIONS'
    })

    # Preenche os NaNs com zero
    df = df.fillna(0)
    
    return df

#### Recuperar urls dos dados

Para permitirmos que o leitor acesse a íntegra dos discursos destacados, vamos recuperar o link de cada discurso, que está disponível nos metadados do diretório `table`.

In [24]:
def get_urls(row, metadata):
    
    '''
    Função para ser rodada via df.apply, que
    usa o nome de um arquivo para recuperar
    o link para o conteúdo original.
    '''
    
    i = row.name
    
    url = metadata.loc[i, 'Discurso']
    
    return pd.Series({
        'URL' : url
    })
    
    

#### Parsear json

In [25]:
def parse_json_chart(df, df_t):

    '''
    PARÂMETROS:
    
    df -> O dataframe original, com todas as declarações, já
    verificado manualmente e com as colunas 'IS_MATCH' e
    'HIGHLIGHT' presentes.

    df_t -> O dataframe que já passou pelo resample,
    que é feito na função resample_df(()
    '''
    
    array = [ ]
    
    for index, row in df_t.iterrows():

        # Seleciona as votações que ocorreram no dia
        this_year  = row.SESSION_DATE.year
        this_month = row.SESSION_DATE.month
        sliced_df  = df[ f"{this_year}-{this_month}" ]

        # Filta para manter apenas aquelas em que houve uso de palavra-chave
        sliced_df = sliced_df[ sliced_df.IS_MATCH == 1 ]

        # Reordena de mais recente para mais tardia
        sliced_df = sliced_df.sort_index().reset_index()

        
        obj = { }

        obj["year"]       = int(row.SESSION_DATE.year)
        obj["month"]      = int(row.SESSION_DATE.month)
        obj["total"]      = int(row.ALL_MENTIONS)
        obj["highlights"] = int(row.IS_MATCH)
        obj["speeches"]   = [ ]
        
        for i in range(obj["highlights"]):
            inner_obj = { }
            inner_obj["excerpt"] = sliced_df.loc[i, 'HIGHLIGHT']
            inner_obj["session"] = sliced_df.loc[i, 'SESSION_TYPE']
            inner_obj["date"]    = str( sliced_df.loc[i, 'SESSION_DATE'] )[:10]
            inner_obj["link"]    = str( sliced_df.loc[i, 'URL'])

            obj["speeches"].append(inner_obj)

        array.append(obj)
        
    return array

#### Executar operações

In [31]:
def run_script_chart():
    
    path = "../data/csvs/DISCURSOS_CHECADOS_MANUALMENTE.csv"
    df = pd.read_csv(path)
    
     # Remove dados de discursos em comissão
    df = df[ df.SESSION_TYPE != 'COMISSÃO ESPECIAL (FORA DO PLENÁRIO)' ]
    
    # Pega a URL correta
    metadata = pd.read_csv("../data/tables/jair-bolsonaro-plenario-metadata.csv")
    df["URL"] = df.apply( get_urls, args = (metadata,), axis = 1 )
    
    # Passa a data do pronunciamento para o index
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE)
    df = df.set_index("SESSION_DATE")
    
    # Faz o resample temporal
    df_t = resample_df(df)

    array = parse_json_chart(df, df_t)
        
    with open('../data/jsons/data.json', 'w') as fp:
        json.dump(array, fp, indent = 2)
        
    pp.pprint(array)

In [32]:
run_script_chart()

[{'highlights': 0, 'month': 1, 'speeches': [], 'total': 2, 'year': 2001},
 {'highlights': 0, 'month': 2, 'speeches': [], 'total': 4, 'year': 2001},
 {'highlights': 1,
  'month': 3,
  'speeches': [{'date': '2001-03-15',
                'excerpt': 'Agora, façam uma pesquisa para ver como era o '
                           'regime militar e como é hoje. Há uma diferença '
                           'enorme! O cidadão tirava seu empréstimo, comprava '
                           'a sua casinha, tinha esperança, sabia o que ia '
                           'comer no dia seguinte. Hoje em dia, não. Agora, '
                           'quem está sendo torturado hoje em dia são os '
                           'militares.',
                'link': 'https://www.camara.leg.br/internet/sitaqweb/TextoHTML.asp?etapa=5&nuSessao=149.1.55.O&nuQuarto=11&nuOrador=2&nuInsercao=0&dtHorarioQuarto=09:20&sgFaseSessao=BC&Data=11/06/2015&txApelido=JAIR '
                        'BOLSONARO, PP-RJ&txFaseSessao=Brev

                        'Dia&txTipoSessao=Deliberativa Extraordinária - '
                        'CD&dtHoraQuarto=13:24&txEtapa=',
                'session': 'BREVES COMUNICACOES'}],
  'total': 1,
  'year': 2004},
 {'highlights': 1,
  'month': 2,
  'speeches': [{'date': '2004-02-18',
                'excerpt': 'Os velhos petistas mentiram por 20 anos para '
                           'chegar ao poder. Teimam em falar na ditadura '
                           'militar, mas ela está servindo, e muito, para '
                           'polpudas indenizações aos clientes do Sr. '
                           'Greenhalgh. Quem tem vovó e quer dizer que ela foi '
                           'torturada é só procurar seu escritório de '
                           'advocacia, em São Paulo',
                'link': 'https://www.camara.leg.br/internet/sitaqweb/TextoHTML.asp?etapa=5&nuSessao=287.2.54.O&nuQuarto=49&nuOrador=1&nuInsercao=33&dtHorarioQuarto=10:36&sgFaseSessao=OD&Data=01/11/2012&txApeli

                           'este País em nome da autoridade, do respeito e da '
                           'família, porque os militares não querem assumir '
                           'isso daí.',
                'link': 'https://www.camara.leg.br/internet/sitaqweb/TextoHTML.asp?etapa=5&nuSessao=335.2.54.O&nuQuarto=85&nuOrador=1&nuInsercao=0&dtHorarioQuarto=16:48&sgFaseSessao=GE&Data=04/12/2012&txApelido=JAIR '
                        'BOLSONARO, PP-RJ&txFaseSessao=Grande '
                        'Expediente&txTipoSessao=Ordinária - '
                        'CD&dtHoraQuarto=16:48&txEtapa=',
                'session': 'BREVES COMUNICACOES'},
               {'date': '2009-03-25',
                'excerpt': 'Vocês querem inverter a história! A luta armada '
                           'não veio depois do AI-5; o AI-5 veio depois da '
                           'luta armada! Vocês querem que não se torne '
                           'público, por exemplo, o latrocínio praticado por '
   

                           'projeto, porque, em seu art. 4º, ele desborda a '
                           'Lei da Anistia e permite a prisão de militares. '
                           'Isso é uma covardia. Estão cisalhando as Forças '
                           'Armadas.',
                'link': 'https://www.camara.leg.br/internet/sitaqweb/TextoHTML.asp?etapa=5&nuSessao=136.2.54.O&nuQuarto=5&nuOrador=2&nuInsercao=0&dtHorarioQuarto=09:08&sgFaseSessao=BC&Data=24/05/2012&txApelido=JAIR '
                        'BOLSONARO, PP-RJ&txFaseSessao=Breves '
                        'Comunicações&txTipoSessao=Extraordinária - '
                        'CD&dtHoraQuarto=09:08&txEtapa=',
                'session': 'ABERTURA'},
               {'date': '2011-09-14',
                'excerpt': 'Por favor, leiam o projeto de lei que trata da '
                           'Comissão da Verdade. Por favor, leiam. Muitos '
                           'colegas não o leram ainda. Eu tenho aqui a cópia '
        

                        'BOLSONARO, PP-RJ&txFaseSessao=Grande '
                        'Expediente&txTipoSessao=Deliberativa Ordinária - '
                        'CD&dtHoraQuarto=16:52&txEtapa=',
                'session': 'BREVES COMUNICACOES'}],
  'total': 6,
  'year': 2013},
 {'highlights': 4,
  'month': 9,
  'speeches': [{'date': '2013-09-05',
                'excerpt': 'O jornal O Globo acabou de assumir uma posição que '
                           'alguns dizem ser corajosa. Para mim é um grande '
                           'retrocesso: "Apoio editorial ao Golpe de 1964 foi '
                           'um erro". Essa posição não condiz com a verdade. '
                           'Quero apelar daqui aos filhos do honrado Roberto '
                           'Marinho: o jornalismo se faz com verdades. No '
                           'editorial de 2 de abril de 1964, Roberto Marinho '
                           'se insurge aqui dizendo que o Brasil vivia uma '
                   

A visualização também exige um segundo JSON, com o texto dos destaques, que será gerado abaixo.

In [20]:
def parse_json_highlights():
    
    df = pd.read_csv("../data/csvs/sneakpeek.csv")
    df["DATE"] = pd.to_datetime(df.DATE)
    
    array = [ ]
    
    for index, row in df.iterrows():
        
        obj = { }
        
        obj["year"]   = row.DATE.year
        obj["month"]  = row.DATE.month
        obj["speech"] = row.SPEECH_INDEX
        obj["text"]   = row.TEXT
        
        array.append(obj)
        
    with open('../data/jsons/sneakpeeks.json', 'w') as fp:
        json.dump(array, fp, indent = 2)
    
    pp.pprint(array)

In [21]:
parse_json_highlights()

[{'month': 8,
  'speech': 0,
  'text': 'Está claro que a âncora do real é o arrocho salarial e a humilhação '
          'dos menos favorecidos. Essa é uma atitude covarde. Não poderíamos '
          'esperar outra coisa do cidadão Fernando Henrique Cardoso, que, '
          'infelizmente, o regime militar não mandou para o espaço.',
  'year': 2001},
 {'month': 3,
  'speech': 0,
  'text': 'O regime militar, tão condenado, deixou vivo Aloysio Nunes '
          'Ferreira, portanto não era tão violento como dizem. [...] Por falar '
          'em violência, não existe hoje classe que esteja sendo mais '
          'violentada do que a dos militares das Forças Armadas.',
  'year': 2002},
 {'month': 7,
  'speech': 1,
  'text': 'Ouvi há pouco no corredor as pessoas pedindo a volta da ditadura e '
          'das cassações para varrer da Pátria esses [petistas] traidores, '
          'mandando-os para o jângal, para o valão. É isso que eles merecem: '
          'ditadura já!',
  'year': 2003},
 {